## Goal oriented service composition

Given a user query in natural language, the solution can generate sequence of function calls from openAPI docstring. 

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: Arrhythmia Identification from ECG. 

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint) 
3. [Clean-up](#3.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

# 1. Subscribe to the model package

To subscribe to the algorithm:
1. Open the model package listing page Synthetic-chest-X-ray-image-generator.
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn=''

### 2. Prepare dataset

In [2]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import urllib.request
import numpy as np

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()
bucket = sagemaker_session.default_bucket()
bucket

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "planner-agent"

content_type = "application/zip"

real_time_inference_instance_type = "ml.m5.2xlarge"
batch_transform_inference_instance_type = "ml.m5.2xlarge"

#### A. Create an endpoint

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session, content_type)

In [6]:
#Create a deployable model from model package
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn, 
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [ ]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

#### B. Create input payload

#### Instructions

    1) The payload should be in zip format.

In [8]:
file_name = "input/input_zip.zip"

#### C. Perform real-time inference

In [9]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'application/zip' --region us-east-2 output.zip

{
    "ContentType": "application/zip",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

- The output file (in zip format) contains the following files:

    1. 'output.zip': A folder containing sequence of function calls that has to be made is generated by the model.

In [10]:
import os 
from zipfile import *

file_path = os.getcwd()
file_name_ = 'output.zip'

file_object = open(file_name_,'rb')
z = ZipFile(file_object)
file_names = []
for name in z.namelist():
    z.extract(name,file_path)
    file_names.append(name)

file_object.close()


In [11]:
with open('plan.json', 'r') as f:
    plan = json.load(f)

In [12]:
print('The plan is as follows: \n', plan['plan'])
print('-'*15)
print('\nThe reason for the plan is as follows: \n', plan['reason_for_plan'])

The plan is as follows: 
 {'step1': {'goal': 'check_balance', 'query': ['Check the account balance from the database.']}, 'step2': {'goal': 'check_payee', 'query': ['Check if Alex is a payee in the system.']}, 'step3': {'goal': 'retrieve_general_information', 'query': ['Retrieve information regarding interest rates on personal loans from the knowledge base.']}, 'step4': {'goal': 'logical_result_comparator', 'query': ['Compare if the account balance retrieved in step 1 is sufficient to transfer $5000 to Alex. If sufficient, transfer $5000 to Alex. If not sufficient, process a loan for the amount needed to transfer $5000 to Alex.']}}
---------------

The reason for the plan is as follows: 
 The plan first checks the account balance and verifies if Alex is a payee. It then retrieves general information about interest rates on personal loans. Finally, it compares the account balance with the amount to be transferred ($5000) and either transfers the money to Alex if the balance is sufficien

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [11]:
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Clean-up

#### A. Delete the model

In [ ]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

